In the newsletter, following are the steps to building a RAG pipeline:

Step 1: Take an inbound query and deconstruct it into relevant concepts  
Step 2: Collect similar concepts from your data store  
Step 3: Recombine these concepts with your original query to build a more relevant, authoritative answer.  
Let's see how is the implementation in this project

# Step 1: Take an inbound query and deconstruct it into relevant concepts

We can take a look at the `query_pipeline.py` file, specifically within the `configure_query_pipeline` function:

```python
def configure_query_pipeline(*, index: VectorStoreIndex, llm: OpenAI) -> QueryPipeline:
    """Configure and set up the query pipeline"""
    text_qa_chat_template = ChatPromptTemplate.from_messages(_chat_template_messages)
    query_pipeline = QueryPipeline()

    retriever = index.as_retriever(similarity_top_k=_TOP_K_RETRIEVAL)
    summarizer = TreeSummarize(
        llm=llm, streaming=True, summary_template=text_qa_chat_template
    )

    query_pipeline.add_modules(
        {
            "input": InputComponent(),
            "retriever": retriever,
            "summarizer": summarizer,
        }
    )
    query_pipeline.add_link("input", "retriever")
    query_pipeline.add_link("input", "summarizer", dest_key="query_str")
    query_pipeline.add_link("retriever", "summarizer", dest_key="nodes")

    return query_pipeline
```

It takes the inbound query using InputComponent library and deconstructs it using QueryPipeline library.
Those classes are provided by llama_index.core.query_pipeline (see llama_index.core.query_pipeline note for more detail)

`InputComponent`is responsible for taking the raw query input and preparing it for further processing.
`QueryPipeline` is to chain different components to create workflows for processing queries.